In [3]:
import threading
import pandas as pd
import pandas_datareader.data as web
import datetime
from sklearn import linear_model
from time import sleep
import joblib

In [5]:
# this class is to give colors to the print statements
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


# This function will give the price after the delay of sleep_time units of seconds
def getprice(label="WIPRO.NS", sleep_time=1):
    # Pause for sleep_time number of seconds
    sleep(sleep_time)
    # Using yahoo finance to get the stock market live data
    quote = web.get_quote_yahoo(label)
    price = quote["price"].values[0]
    current_time = datetime.datetime.now()
    return tuple([price, current_time])


# This function trains the model using the input data(dataframe)
def train(input):
    print("\nModel updating...", end=" ")
    # We take last column of the features as target and rest are taken as attributes
    featureMat = input.iloc[:, : len(input.columns) - 1]
    label = input[input.columns[-1]]
    # Here we are using linear regression model
    model = linear_model.LinearRegression()
    model.fit(featureMat, label)
    # Model is being written on the hard drive
    joblib.dump(model, "modelLR.pkl")
    print("[Completed]")

# Training over  
Increase the values of these variables to get improved results
but if we will increase them then we'll have to wait for (number_of_features X training_record_criteran) X steep_time units seconds for first training


In [7]:
number_of_features = 5  # This indicates how many columns the dataframe will have.
training_record_criterian = 5  # This decides how frequently the model will update [5 new features -> retrain the model]
number_of_predictions = 3  # Tells how many prediction in series you want

In [6]:
data = pd.DataFrame(columns=range(number_of_features))  # creating an empty dataframe

predict_input = list()

while 1:

    feature = list()  # stores the features for a single record for dataframe

    for i in range(number_of_features):

        price = getprice()[0]
        feature.append(price)
        predict_input.append(price)

        try:  # this will throw exception in two cases:
            # 1> model is not yet trained and saved
            # 2> model prediction is not working.

            first_predict = True  # flag for detecting the first prediction in predicted series
            model = joblib.load("modelLR.pkl")  # trying to open the saved model (can throw exception)
            print("")
            inputlist = predict_input.copy()  # copying the list to make the prediction if model is ready
            #   printing latest 3 prices
            for feature_value in inputlist[-(3):]:
                print(f"{bcolors.WARNING} --> ", int(feature_value * 100) / 100, end=" ")
            #   taking the latest price
            price = getprice(sleep_time=0)[0]
            #   Starting the predictions
            for i in range(number_of_predictions):
                pre_price = model.predict([inputlist[-(number_of_features - 1):]])
                #   printing the predicted values one by one in the series
                print(f"{bcolors.OKBLUE} --> ", int(pre_price[0] * 100) / 100, end=" ")
                #   This block will only run for the first prediction in series
                if first_predict:
                    # When prediction tells about increase in price
                    if pre_price[0] - inputlist[-1] > 0:
                        print(f"{bcolors.OKGREEN}  \u2191", end="")
                        #   Calculating the % of increase the program predicts and printing.
                        print(f"{bcolors.BOLD}[", int((pre_price[0] - price) * 1000000 / price) / 10000, "%] ", end=" ")
                        print(f"{bcolors.OKCYAN} Actual: ", price, end="")
                    # When prediction says that no change will happen
                    elif pre_price[0] - inputlist[-1] == 0:
                        print(f"{bcolors.HEADER} \u2022", end="")
                        print(f"{bcolors.BOLD}[", int((pre_price[0] - price) * 1000000 / price) / 10000, "%] ", end=" ")
                        print(f"{bcolors.OKCYAN} Actual: ", price, end="")
                    # When prediction is about decrease in price
                    else:
                        print(f"{bcolors.FAIL}  \u2193", end="")
                        print(f"{bcolors.BOLD}[", int(-(pre_price[0] - price) * 1000000 / price) / 10000, "%] ",
                              end=" ")
                        print(f"{bcolors.OKCYAN} Actual: ", price, end="")
                    # Next statement talk about what happened actually
                    if price - inputlist[-1] > 0:
                        print(f"{bcolors.OKGREEN}  \u2191", end=" ")
                    elif price - inputlist[-1] == 0:
                        print(f"{bcolors.HEADER} \u2022", end="")
                    else:
                        print(f"{bcolors.FAIL}  \u2193", end=" ")

                    first_predict = False


                #   pushing the predicted price in the back of the input array..
                #   it will be used in predicting next element in the series
                inputlist.append(pre_price[0])
        except:

            print("Please Wait while the model is getting ready...")
    #   Adding the feature in the dataframe
    data.loc[len(data.index)] = feature
    #   If number of elements present in the dataframe is multiple of training record criterian, the retraining
    if len(data.index) % training_record_criterian == 0:
        # print(data)
        #   training in separate thread
        trainer = threading.Thread(target=train, args=(data,))
        trainer.start()
        trainer.join()


 -->  674.8 Please Wait while the model is getting ready...

 -->  674.8  -->  674.8 Please Wait while the model is getting ready...

 -->  674.8  -->  674.8  -->  674.8 Please Wait while the model is getting ready...

 -->  674.8  -->  674.8  -->  674.8  -->  674.78   ↓[ 0.0018 %]   Actual:  674.8 • -->  674.77  -->  674.76 
 -->  674.8  -->  674.8  -->  674.8  -->  674.78   ↓[ 0.0092 %]   Actual:  674.85  ↑  -->  674.77  -->  674.76 
 -->  674.8  -->  674.8  -->  674.85  -->  674.83   ↓[ -0.0349 %]   Actual:  674.6  ↓  -->  674.81  -->  674.84 
 -->  674.8  -->  674.85  -->  674.6  -->  674.58   ↓[ 0.0313 %]   Actual:  674.8  ↑  -->  674.65  -->  674.43 
 -->  674.85  -->  674.6  -->  674.8  -->  674.85   ↑[ 0.0306 %]   Actual:  674.65  ↓  -->  674.61  -->  674.75 
 -->  674.6  -->  674.8  -->  674.65  -->  674.41   ↓[ 0.0499 %]   Actual:  674.75  ↑  -->  674.57  -->  674.47 
 -->  674.8  -->  674.65  -->  674.75  -->  674.9   ↑[ 0.023 %]   Actual:  674.75 • -->  674.75  -->  674.79

 -->  675.1  -->  675.1  -->  675.05  -->  674.87   ↓[ 0.0264 %]   Actual:  675.05 • -->  -17970698.09  -->  -123587962.52 
 -->  675.1  -->  675.05  -->  675.1  -->  -17970697.52   ↓[ 2662031.1984 %]   Actual:  675.1 • -->  -41333483.19  -->  -6459404728884613.0 
Model updating... [Completed]

 -->  675.05  -->  675.1  -->  675.1  -->  675.1   ↑[ 0.0006 %]   Actual:  675.1 • -->  675.1  -->  675.11 
 -->  675.1  -->  675.1  -->  675.1  -->  675.1   ↑[ 0.0003 %]   Actual:  675.1 • -->  675.11  -->  675.11 
 -->  675.1  -->  675.1  -->  675.1  -->  675.11   ↑[ 0.0017 %]   Actual:  675.1 • -->  675.11  -->  675.11 
 -->  675.1  -->  675.1  -->  675.1  -->  675.11   ↑[ 0.0017 %]   Actual:  675.1 • -->  675.11  -->  675.11 
 -->  675.1  -->  675.1  -->  675.1  -->  675.11   ↑[ 0.0017 %]   Actual:  675.1 • -->  675.11  -->  675.11 
 -->  675.1  -->  675.1  -->  675.1  -->  675.11   ↑[ 0.0091 %]   Actual:  675.05  ↓  -->  675.11  -->  675.11 
 -->  675.1  -->  675.1  -->  675.05  -->  675.09

 -->  674.75  -->  674.9  -->  674.75  -->  674.79   ↑[ 0.0069 %]   Actual:  674.75 • -->  674.81  -->  674.84 
 -->  674.9  -->  674.75  -->  674.75  -->  674.79   ↑[ 0.0066 %]   Actual:  674.75 • -->  674.83  -->  674.81 
Model updating... [Completed]

 -->  674.75  -->  674.75  -->  674.75  -->  674.82   ↑[ 0.0115 %]   Actual:  674.75 • -->  674.81  -->  674.8 
 -->  674.75  -->  674.75  -->  674.75  -->  674.77   ↑[ 0.0036 %]   Actual:  674.75 • -->  674.78  -->  674.79 
 -->  674.75  -->  674.75  -->  674.75  -->  674.77   ↑[ 0.0036 %]   Actual:  674.75 • -->  674.78  -->  674.79 
 -->  674.75  -->  674.75  -->  674.75  -->  674.77   ↑[ 0.0036 %]   Actual:  674.75 • -->  674.78  -->  674.79 
 -->  674.75  -->  674.75  -->  674.75  -->  674.77   ↑[ 0.0036 %]   Actual:  674.75 • -->  674.78  -->  674.79 
 -->  674.75  -->  674.75  -->  674.75  -->  674.77   ↑[ 0.0036 %]   Actual:  674.75 • -->  674.78  -->  674.79 
 -->  674.75  -->  674.75  -->  674.75  -->  674.77   ↑[ 0.0036 %]  

 -->  674.95  -->  674.95  -->  674.95  -->  674.96   ↑[ 0.0167 %]   Actual:  674.85  ↓  -->  674.96  -->  674.97 
 -->  674.95  -->  674.95  -->  674.85  -->  674.91   ↑[ 0.0025 %]   Actual:  674.9  ↑  -->  674.93  -->  674.93 
Model updating... [Completed]

 -->  674.95  -->  674.85  -->  674.9  -->  674.94   ↑[ 0.0138 %]   Actual:  674.85  ↓  -->  674.95  -->  674.93 
 -->  674.85  -->  674.9  -->  674.85  -->  674.9   ↑[ 0.0154 %]   Actual:  674.8  ↓  -->  674.9  -->  674.91 
 -->  674.9  -->  674.85  -->  674.8  -->  674.85   ↑[ 0.0085 %]   Actual:  674.8 • -->  674.89  -->  674.89 
 -->  674.85  -->  674.8  -->  674.8  -->  674.86   ↑[ 0.01 %]   Actual:  674.8 • -->  674.88  -->  674.87 
 -->  674.8  -->  674.8  -->  674.8  -->  674.84   ↑[ 0.0069 %]   Actual:  674.8 • -->  674.85  -->  674.85 
 -->  674.8  -->  674.8  -->  674.8  -->  674.83   ↑[ 0.0047 %]   Actual:  674.8 • -->  674.84  -->  674.85 
 -->  674.8  -->  674.8  -->  674.8  -->  674.83   ↑[ 0.0047 %]   Actual:  674.

 -->  675.4  -->  675.25  -->  675.35  -->  675.34   ↓[ -0.0213 %]   Actual:  675.2  ↓  -->  675.32  -->  675.28 
 -->  675.25  -->  675.35  -->  675.2  -->  675.24   ↑[ -0.0149 %]   Actual:  675.35  ↑  -->  675.24  -->  675.25 
Model updating... [Completed]

 -->  675.35  -->  675.2  -->  675.35  -->  675.27   ↓[ 0.0117 %]   Actual:  675.35 • -->  675.24  -->  675.22 
 -->  675.2  -->  675.35  -->  675.35  -->  675.28   ↓[ -0.0052 %]   Actual:  675.25  ↓  -->  675.24  -->  675.24 
 -->  675.35  -->  675.35  -->  675.25  -->  675.22   ↓[ -0.0115 %]   Actual:  675.15  ↓  -->  675.23  -->  675.22 
 -->  675.35  -->  675.25  -->  675.15  -->  675.2   ↑[ 0.0074 %]   Actual:  675.15 • -->  675.2  -->  675.17 
 -->  675.25  -->  675.15  -->  675.15  -->  675.18   ↑[ 0.0044 %]   Actual:  675.15 • -->  675.16  -->  675.14 
 -->  675.15  -->  675.15  -->  675.15  -->  675.15   ↑[ 0.0074 %]   Actual:  675.1  ↓  -->  675.13  -->  675.12 
 -->  675.15  -->  675.15  -->  675.1  -->  675.1   ↑[ 0.00

 -->  675.15  -->  675.15  -->  675.15  -->  675.12   ↓[ 0.0181 %]   Actual:  675.25  ↑  -->  675.11  -->  675.1 
 -->  675.15  -->  675.15  -->  675.25  -->  675.17   ↓[ -0.0258 %]   Actual:  675.0  ↓  -->  675.15  -->  675.13 
Model updating... [Completed]

 -->  675.15  -->  675.25  -->  675.0  -->  675.07   ↑[ -0.0034 %]   Actual:  675.1  ↑  -->  675.08  -->  675.09 
 -->  675.25  -->  675.0  -->  675.2  -->  675.14   ↓[ 0.0083 %]   Actual:  675.2 • -->  675.13  -->  675.1 
 -->  675.0  -->  675.2  -->  675.15  -->  675.14   ↓[ 0.0159 %]   Actual:  675.25  ↑  -->  675.1  -->  675.11 
 -->  675.2  -->  675.15  -->  675.25  -->  675.15   ↓[ -0.0007 %]   Actual:  675.15  ↓  -->  675.15  -->  675.14 
 -->  675.15  -->  675.25  -->  675.15  -->  675.15   ↑[ 0.0228 %]   Actual:  675.0  ↓  -->  675.14  -->  675.14 
 -->  675.25  -->  675.15  -->  675.0  -->  675.07   ↑[ 0.0107 %]   Actual:  675.0 • -->  675.09  -->  675.08 
 -->  675.15  -->  675.0  -->  675.0  -->  675.06   ↑[ -0.013 %] 

KeyboardInterrupt: 